In [5]:
from src.constants import *
from src.model.my_model import MyModel
from src.model.baselines import UNet
import numpy as np
from torch.profiler import profile, record_function, ProfilerActivity

In [6]:
example2 = torch.rand(size=[2, 1, 128, 128, 128])
mask = torch.rand(size=[2, 3, 128, 128, 128])
loss_fn = monai.losses.DiceCELoss()

In [7]:
model = MyModel(in_channels=1,
                 out_channels=3,
                 lower_channels=32,
                 big_channel=32,
                 patch_size=8,
                 embed_dim=256,
                 skip_transformer=True)
optimizer = torch.optim.Adam(params=model.parameters(), lr=3e-4)

# model = UNet.cuda()

In [ ]:
%%time
out = model(example2)
l = loss_fn(out, mask)
l.backward()
optimizer.step()

In [12]:
inputs = torch.rand(size=[4, 1, 256, 256, 256]).cuda()

with profile(activities=[ProfilerActivity.CUDA],
        profile_memory=True, record_shapes=True) as prof:
    model(inputs)

print(prof.key_averages().table(sort_by="cuda_memory_usage"))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                       aten::empty_like         3.23%     279.000us         8.83%     762.000us       7.327us           0 b           0 b      31.64 Gb     514.00 Mb           104  
                                    aten::empty_strided         5.70%     492.000us         5.70%     492.000us       4.731us           0 b           0 b      31.64 Gb      31.64 Gb           104  
         

STAGE:2023-04-25 23:21:58 435553:435553 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-04-25 23:21:58 435553:435553 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-04-25 23:21:58 435553:435553 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
